In [1]:
import json
import os
from collections import Counter
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [271]:
review = {}
with open('review_ko_dict.json','r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [272]:
review['10010172']

[[5, '정말 편하게 지낼 수 있고 매일 즐길 수 있는 곳이입니다. 추천합니다.'],
 [5, '전반적으로 모두 마음에 들었습니다. 지내기에 불편한점이 없습니다'],
 [5,
  '가성비넘나좋네요 원래 일주일예약했다가 방이 괜찮아서 2일 추가해서 살았습니다 너무좋은말만해서 최상급의 기대까진 마세요 다만 이 가격에 이정도 방은 찾기힘들것같아요']]

In [273]:
review_ko = {}

for home_id,rev_lst in review.items():
    review_ko[home_id] = []
    for rev in rev_lst:
        if  (rev[0] > 2) & (rev[0] < 4) :
            review_ko[home_id].append([rev[0],rev[1]])
        else:
            continue

In [274]:
review_total = []
for home_id,rev_lst in review_ko.items():
    for rev in rev_lst :
        if len(rev) > 1 :
            review_total.append(rev)
            
print('number of reviews : ', len(review_total))

number of reviews :  3337


In [275]:
del review_ko

In [276]:
review_total[0]

[3,
 '위치 하나는 정말 최고였어요!\n빠른 응답도 좋았습니다. 화장실은 다만 세탁기때문에 좀 많이 좁더라구요 :)\n다리미까지 구비되어 있는 센스 좋았어요.']

In [277]:
sen = review_total[0]
re.sub('[a-zA-Z]' , '', sen[1])

'위치 하나는 정말 최고였어요!\n빠른 응답도 좋았습니다. 화장실은 다만 세탁기때문에 좀 많이 좁더라구요 :)\n다리미까지 구비되어 있는 센스 좋았어요.'

In [214]:
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
hangul.sub('', sen[1])

'위치 하나는 정말 최고였어요빠른 응답도 좋았습니다 화장실은 다만 세탁기때문에 좀 많이 좁더라구요 다리미까지 구비되어 있는 센스 좋았어요'

In [5]:
from konlpy.tag import Twitter, Kkma, Okt
twitter = Twitter()
kkma = Kkma()
okt = Okt()

/usr/local/lib/python3.5/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [278]:
review_total[0]

[3,
 '위치 하나는 정말 최고였어요!\n빠른 응답도 좋았습니다. 화장실은 다만 세탁기때문에 좀 많이 좁더라구요 :)\n다리미까지 구비되어 있는 센스 좋았어요.']

In [279]:
review_split_sen = review_total.copy()
for rev in review_split_sen :
    rev[1] = kkma.sentences(rev[1])

In [216]:
review_split_sen[0]

[3,
 ['위치 하나는 정말 최고였어요!',
  '빠른 응답도 좋았습니다.',
  '화장실은 다만 세탁기 때문에 좀 많이 좁더라구요',
  ':) 다리 미까지 구비되어 있는 센스 좋았어요.']]

In [280]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

for rev in review_split_sen : 
    sen_lst = []
    for r in rev[1] :
        sen = hangul.sub('', r)
        sen = sen.strip().replace("ㅎ", "").replace("ㅠ", "").replace("ㅜ", "")
        if len(sen) > 1 :
            sen_lst.append(sen)
        rev[1] = sen_lst    

In [281]:
review_split_sen[0]

[3,
 ['위치 하나는 정말 최고였어요',
  '빠른 응답도 좋았습니다',
  '화장실은 다만 세탁기 때문에 좀 많이 좁더라구요',
  '다리 미까지 구비되어 있는 센스 좋았어요']]

In [282]:
with open('review_split_sen.json','w',encoding='utf-8') as fp:
        json.dump(review_split_sen,fp)

In [200]:
ex = '먹다 먹는다 먹었다 먹였다 먹었더라구요'

print(twitter.morphs(ex))
print(kkma.morphs(ex))
print(okt.morphs(ex))

['먹다', '먹는다', '먹었다', '먹였다', '먹었더라구요']
['먹', '다', '먹', '는', '다', '먹', '었', '다', '먹이', '었', '다', '먹', '었', '더', '라구요']
['먹다', '먹는다', '먹었다', '먹였다', '먹었더라구요']


In [202]:
kkma.pos(ex)

[('먹', 'VV'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('는', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('다', 'ECS'),
 ('먹이', 'VV'),
 ('었', 'EPT'),
 ('다', 'ECS'),
 ('먹', 'VV'),
 ('었', 'EPT'),
 ('더', 'EPT'),
 ('라구요', 'EFN')]

In [13]:
with open('review_split_sen.json') as data_file:    
    review_split_sen = json.load(data_file)

In [14]:
for rev in review_split_sen : 
    sen_lst = []
    for r in rev[1] :
        sen = kkma.pos(r)
        sen_lst.append(sen)
    rev[1] = sen_lst 

In [285]:
kkma.tagset # 'Noun': '명사' / 'Adjective': '형용사' /  'Verb': '동사'

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [203]:
review_split_sen[0] # twitter

[3,
 [[('위치', 'Noun'),
   ('하나', 'Noun'),
   ('는', 'Josa'),
   ('정말', 'Noun'),
   ('최고', 'Noun'),
   ('였어요', 'Verb')],
  [('빠른', 'Adjective'), ('응답', 'Noun'), ('도', 'Josa'), ('좋았습니다', 'Adjective')],
  [('화장실', 'Noun'),
   ('은', 'Josa'),
   ('다만', 'Noun'),
   ('세탁기', 'Noun'),
   ('때문', 'Noun'),
   ('에', 'Josa'),
   ('좀', 'Noun'),
   ('많이', 'Adverb'),
   ('좁더라구요', 'Adjective')],
  [('다리', 'Noun'),
   ('미까지', 'Adjective'),
   ('구비', 'Noun'),
   ('되어', 'Verb'),
   ('있는', 'Adjective'),
   ('센스', 'Noun'),
   ('좋았어요', 'Adjective')]]]

In [284]:
review_split_sen[0] # kkma

[3,
 [[('위치', 'NNG'),
   ('하나', 'NR'),
   ('는', 'JX'),
   ('정말', 'MAG'),
   ('최고', 'NNG'),
   ('이', 'VCP'),
   ('었', 'EPT'),
   ('어요', 'EFN')],
  [('빠르', 'VA'),
   ('ㄴ', 'ETD'),
   ('응답', 'NNG'),
   ('도', 'JX'),
   ('좋', 'VA'),
   ('았', 'EPT'),
   ('습니다', 'EFN')],
  [('화장실', 'NNG'),
   ('은', 'JX'),
   ('다만', 'MAG'),
   ('세탁기', 'NNG'),
   ('때문', 'NNB'),
   ('에', 'JKM'),
   ('좀', 'MAG'),
   ('많이', 'MAG'),
   ('좁', 'VA'),
   ('더', 'EPT'),
   ('라구요', 'EFN')],
  [('다리', 'NNG'),
   ('미', 'NNG'),
   ('까지', 'JX'),
   ('구비', 'NNG'),
   ('되', 'XSV'),
   ('어', 'ECD'),
   ('있', 'VXV'),
   ('는', 'ETD'),
   ('센스', 'NNG'),
   ('좋', 'VA'),
   ('았', 'EPT'),
   ('어요', 'EFN')]]]

In [290]:
for sen in review_split_sen[0][1] :
    for (word,pos) in sen :
        if pos in ['NNG','VA'] :
            print(word)

위치
최고
빠르
응답
좋
화장실
세탁기
좁
다리
미
구비
센스
좋


In [69]:
#  'NN': '명사', 'NNG': '보통명사', 'NNP': '고유명사', 'NP': '대명사',
#  'VA': '형용사', 'VV': '동사',
#  'XR': '어근'

for rev in review_split_sen : 
    sen_lst = []
    for sen in rev[1] :
        w_lst = []
        for (word,pos) in sen :
            if pos in ['NN','NNG','NNP','NP','VA','VV','XR'] :
                w_lst.append(word)
        sen_lst.append(w_lst)
        rev[1] = sen_lst

In [71]:
with open('review_NV.json','w',encoding='utf-8') as fp:
        json.dump(review_split_sen,fp)